In [2]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import os
# from functools import wraps
# from IPython.display import clear_output
import time

import sys
sys.path.insert(0, '..')
from ugi_rxn_mapper import ugi_rxn_mapper
from chemprop.args import PredictArgs
from chemprop.train import make_predictions

In [13]:
# class silence_stdout:
#     def __enter__(self):
#         self._original_stdout = sys.stdout
#         sys.stdout = open(os.devnull, 'w')

#     def __exit__(self, exc_type, exc_val, exc_tb):
#         sys.stdout.close()
#         sys.stdout = self._original_stdout

#     def __call__(self, func):
#         @wraps(func)
#         def wrapper(*args, **kwargs):
#             with self:
#                 return func(*args, **kwargs)
#         return wrapper
# import contextlib
# import io

# def silence_stdout(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         with contextlib.redirect_stdout(io.StringIO()):
#             return func(*args, **kwargs)
#     return wrapper

In [25]:
def get_ugi_prod(smi_list) -> str:  # generate 4c4c ugi prod from list of 4 bbs
    ugi_4c_4c_smarts = "[NH2,NH3:5].[CH1:2]=O.[C:1](=O)[OH,O-].[N+:3]#[C-:4]>>[C:1](=O)[N:5][C:2][C+0:4](=O)[N+0:3]"
    ugi_4c_4c_reaction = AllChem.ReactionFromSmarts(ugi_4c_4c_smarts)
    ugi_4c_4c_prod = ugi_4c_4c_reaction.RunReactants([Chem.MolFromSmiles(smi) for smi in smi_list])
    print(ugi_4c_4c_prod)
    return Chem.MolToSmiles(ugi_4c_4c_prod[0][0])


def set_model_args():
    """Load Chemprop model."""
    # define args for chemprop predictor
    args = PredictArgs()
    args.features_generator =  ["rdkit_2d","ifg_drugbank_2","ugi_qmdesc_atom"]
    args.number_of_molecules = 2
    args.gpu = 0
    args.checkpoint_paths = ['/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_2/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_1/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_8/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_6/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_0/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_4/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_9/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_7/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_3/model_0/model.pt', '/home/jnliu/chemprop/benchmark_chemprop/hyper_opt/opt_for_pred/trial_seed_60/fold_5/model_0/model.pt']
    args.no_features_scaling = False
    args.preds_path = "./preds.csv"

    # load model
    # model_objects = load_model(args)
    return args

# @silence_stdout()  
def get_expected_return(args, combination):

    smi_list = [all_bbs_list[i] for i in combination]
    prod_smi = get_ugi_prod(smi_list)
    mapped_rxn = ugi_rxn_mapper([prod_smi])[0]
    rxn_smi = [[mapped_rxn,"FC(F)(F)CO"]]
    score = make_predictions(args, rxn_smi)[0][0]
    return score

In [8]:
df_amine_ace = pd.read_csv('../data/ugi/BBs_with_ace/ace_on_amine.smi')
df_aldehyde = pd.read_csv('../data/ugi/BBs/aldehyde.smi')
df_acid = pd.read_csv('../data/ugi/BBs/acid.smi')
df_isocyanide = pd.read_csv('../data/ugi/BBs/NC.smi')
df_amine_ace.columns = ['smiles']
total_bb_list = df_amine_ace["smiles"].tolist()
print(f"num of amine_ace: {len(df_amine_ace)} \n num of aldehyde: {len(df_aldehyde)} \n num of acid: {len(df_acid)} \n num of isocyanide: {len(df_isocyanide)}")

num of amine_ace: 3 
 num of aldehyde: 937 
 num of acid: 1619 
 num of isocyanide: 1


In [22]:
# load model params
args = set_model_args()

### parse smi file

In [11]:
amine_path = '../data/ugi/BBs_with_ace/ace_on_amine.smi'
aldehyde_path = '../data/ugi/BBs/aldehyde.smi'
acid_path = '../data/ugi/BBs/acid.smi'
isocyanide_path = '../data/ugi/BBs/NC.smi'

with open(amine_path, 'r') as f:
    amine_content = f.readlines()
    amine_smi_list = [line.strip().split()[0] for line in amine_content] 
    amine_name_list = [line.strip().split()[1] for line in amine_content]

with open(aldehyde_path, 'r') as f:
    aldehyde_content = f.readlines()
    aldehyde_smi_list = [line.strip().split()[0] for line in aldehyde_content]
    aldehyde_name_list = [line.strip().split()[1] for line in aldehyde_content]

with open(acid_path, 'r') as f:
    acid_content = f.readlines()
    acid_smi_list = [line.strip().split()[0] for line in acid_content]
    acid_name_list = [line.strip().split()[1] for line in acid_content]

with open(isocyanide_path, 'r') as f:
    isocyanide_content = f.readlines()
    isocyanide_smi_list = [line.strip().split()[0] for line in isocyanide_content]
    isocyanide_name_list = [line.strip().split()[1] for line in isocyanide_content]

all_bbs_list = amine_smi_list + aldehyde_smi_list + acid_smi_list + isocyanide_smi_list
all_idx_list = amine_name_list + aldehyde_name_list + acid_name_list + isocyanide_name_list

In [11]:
print(len(all_idx_list))
print(len(all_bbs_list))

2564
2564


### GA

In [5]:
POPULATION_SIZE = 100
category_sizes = [4,938,1620,2]
population = np.zeros((POPULATION_SIZE, len(category_sizes)), dtype=int)
print(population.shape)

(100, 4)


In [9]:
population = np.zeros((POPULATION_SIZE, len(category_sizes)), dtype=int)
for i in range(len(category_sizes)):
    if i == 0:
        population[:, i] = np.random.choice(category_sizes[i], size=POPULATION_SIZE)
    else:
        population[:, i] = np.random.choice(range(np.sum(category_sizes[:i]), np.sum(category_sizes[:i+1])), size=POPULATION_SIZE)
print(population)

[[   0  108 1668 2562]
 [   1  868 2012 2562]
 [   3  113 1146 2563]
 [   3  448 2526 2563]
 [   3  381 1105 2563]
 [   0  160 2556 2563]
 [   2  898 2463 2562]
 [   2  887 1935 2563]
 [   1  566 1016 2563]
 [   3  114 1479 2562]
 [   3  334 1672 2562]
 [   0  866 1707 2563]
 [   2  508 2480 2563]
 [   1  382 2198 2563]
 [   3  203 2325 2563]
 [   1  287 2411 2563]
 [   1  315  992 2562]
 [   0  618 1131 2563]
 [   2  711 2345 2562]
 [   3  853 1830 2562]
 [   2  294 2490 2562]
 [   3  812 1987 2563]
 [   1  426 1458 2562]
 [   3  276 1755 2563]
 [   1  632 1359 2563]
 [   1  396 1296 2562]
 [   3  446 1434 2562]
 [   3  720 1319 2563]
 [   0  301 1359 2562]
 [   2  838 1923 2563]
 [   0  499 1927 2563]
 [   2  280 1696 2563]
 [   0  417  981 2562]
 [   0  883 2446 2563]
 [   1  518 2437 2562]
 [   2  548 2367 2563]
 [   2  546 1597 2563]
 [   1  771 1683 2562]
 [   3  764 1205 2563]
 [   0  856 2430 2563]
 [   1  320 1612 2563]
 [   1  201 2266 2563]
 [   3  666 2126 2563]
 [   2  192

In [16]:
# print(all_bbs_list[2561])
for individual in population:
    # print([all_bbs_list[i] for i in individual])
    # print(individual)

[   0  108 1668 2562]
[   1  868 2012 2562]
[   3  113 1146 2563]
[   3  448 2526 2563]
[   3  381 1105 2563]
[   0  160 2556 2563]
[   2  898 2463 2562]
[   2  887 1935 2563]
[   1  566 1016 2563]
[   3  114 1479 2562]
[   3  334 1672 2562]
[   0  866 1707 2563]
[   2  508 2480 2563]
[   1  382 2198 2563]
[   3  203 2325 2563]
[   1  287 2411 2563]
[   1  315  992 2562]
[   0  618 1131 2563]
[   2  711 2345 2562]
[   3  853 1830 2562]
[   2  294 2490 2562]
[   3  812 1987 2563]
[   1  426 1458 2562]
[   3  276 1755 2563]
[   1  632 1359 2563]
[   1  396 1296 2562]
[   3  446 1434 2562]
[   3  720 1319 2563]
[   0  301 1359 2562]
[   2  838 1923 2563]
[   0  499 1927 2563]
[   2  280 1696 2563]
[   0  417  981 2562]
[   0  883 2446 2563]
[   1  518 2437 2562]
[   2  548 2367 2563]
[   2  546 1597 2563]
[   1  771 1683 2562]
[   3  764 1205 2563]
[   0  856 2430 2563]
[   1  320 1612 2563]
[   1  201 2266 2563]
[   3  666 2126 2563]
[   2  192 1205 2562]
[   1   40 1589 2562]
[   3  222

In [27]:
# 定义参数
POPULATION_SIZE = 200  # 种群大小
GENERATIONS = 50  # 迭代次数
MUTATION_RATE = 0.1  # 变异率

# 假设我们有1000个商品，分成4个类别，每个类别的商品数量分别是
category_sizes = [len(amine_smi_list), len(aldehyde_smi_list), len(acid_smi_list), len(isocyanide_smi_list)]

# 初始化种群
population = np.zeros((POPULATION_SIZE, len(category_sizes)), dtype=int)
for i in range(len(category_sizes)):
    if i == 0:
        population[:, i] = np.random.choice(category_sizes[i], size=POPULATION_SIZE)
    else:
        population[:, i] = np.random.choice(range(np.sum(category_sizes[:i]), np.sum(category_sizes[:i+1])), size=POPULATION_SIZE)

best_individuals = []

for generation in tqdm(range(GENERATIONS), desc='Generation'):
    
    # 计算适应度
    fitness = np.array([get_expected_return(args, individual) for individual in population])
    print(f"shape of fitness:{fitness.shape}")
    print(fitness)

    # 保存当前代的最优解
    best_individuals.append(population[np.argmax(fitness)])
    # print(best_individuals)

    parents = population[np.random.choice(np.arange(POPULATION_SIZE), size=POPULATION_SIZE, p=fitness/fitness.sum())]
    # print(f"shape of parents:{parents.shape}")
    # print(parents)

    # selection
    np.random.shuffle(parents)
    # print(f"parents after shuffle:{parents}")
    children = np.zeros_like(parents)

    
    for i in range(0, POPULATION_SIZE, 2):  # set step to 2 to generate 2 children per iteration
        crossover_point = np.random.randint(len(category_sizes))    # randomly select crossover point
        children[i, :crossover_point] = parents[i, :crossover_point]    # before crossover point, child1 is the same as parent1
        children[i, crossover_point:] = parents[i+1, crossover_point:]  # after crossover point, child1 is the same as parent2
        children[i+1, :crossover_point] = parents[i+1, :crossover_point]
        children[i+1, crossover_point:] = parents[i, crossover_point:]

    # crossover
    for i in range(POPULATION_SIZE):
        if np.random.random() < MUTATION_RATE:  # 10 % possibility to mutate
            mutation_category = np.random.randint(len(category_sizes))  # randomly a kind of bbs to mutate
            if mutation_category == 0:
                children[i, mutation_category] = np.random.choice(category_sizes[mutation_category])    # randomly select a bb from the category
            else:
                children[i, mutation_category] = np.random.choice(range(np.sum(category_sizes[:mutation_category]), np.sum(category_sizes[:mutation_category+1])))

    # 更新种群
    population = children
    with open("ga_log.log", "a") as f:
        f.write(f"Generation {generation} finished at {time.ctime()}\n")
    clear_output()
    
# 输出最优解
best_individual = population[np.argmax([get_expected_return(args, individual) for individual in population])]
print('Best combination:', best_individual)


Generation:   0%|          | 0/50 [00:00<?, ?it/s]

((<rdkit.Chem.rdchem.Mol object at 0x7efe346764a0>,),)
Loading training args
Setting molecule featurization parameters to default.
Loading data
Validating SMILES
Test size = 1


Moving model to cuda


Moving model to cuda


Moving model to cuda


...

Moving model to cuda


Moving model to cuda





Generation:   0%|          | 0/50 [00:16<?, ?it/s]

Saving predictions to ./preds.csv
Elapsed time = 0:00:03
shape of fitness:(5,)
[2.34301158e-05 1.48759171e-03 7.78663683e-04 1.28799566e-03
 3.18574134e-04]
shape of parents:(5, 4)
[[   2  897 1615 2562]
 [   0  794 1886 2562]
 [   2   14 1438 2562]
 [   2   14 1438 2562]
 [   2   14 1438 2562]]
parents after shuffle:[[   0  794 1886 2562]
 [   2   14 1438 2562]
 [   2  897 1615 2562]
 [   2   14 1438 2562]
 [   2   14 1438 2562]]


IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
best_2000_individuals = np.array(best_individuals)[np.argsort(best_fitnesses)[-2000:]]

# 输出最好的2000个解
for i, individual in enumerate(best_2000_individuals):
    print(f'Best combination {i+1}:', individual)

In [28]:
# read pkl file
import pickle

with open('best_individuals.pkl', 'rb') as f:
    best_individuals = pickle.load(f)

best_individuals